In [15]:
# This notebook will take the housing data scraped earlier and clean/format it for further investigation.
import pandas as pd

pd.options.display.max_columns = None

In [16]:
priceRange = [
    [1, 250000], [250001, 300000], [300001, 350000], [350001, 400000],
    [400001, 450000], [450001, 500000], [500001, 750000], [750001, 1000000],
    [1000001, 2000000], [2000001, 5000000], [5000001, 50000000]
    ]

In [17]:
for i, price in enumerate(priceRange):
    if i == 0:
        df = pd.read_csv(f'housingData_{price[0]}_{price[1]}.csv', index_col=0)
    else:
        df = pd.concat([df, pd.read_csv(f'housingData_{price[0]}_{price[1]}.csv', index_col=0)], axis=0, ignore_index=False)

In [19]:
df.shape

(2722, 636)

In [7]:
del df